# Code for retrieving Spotify playlists

NOTE: The spotify API requires refreshing every 60 minutes, so if code does not work, you might need to rerun the entire notebook

In [1]:
import requests
import pandas as pd
import json

### Enter your client_id and client_secret here:
- Get this from the spotify API
- Reference for the code found below: Steven Morse (2020) https://stmorse.github.io/journal/spotify-api.html

In [2]:
## Enter your own Client_Id and Client_secret ##
CLIENT_ID = 'Enter your client_id here'
CLIENT_SECRET = 'Enter your client_secret here'

In [3]:
AUTH_URL = 'https://accounts.spotify.com/api/token'
BASE_URL = 'https://api.spotify.com/v1/'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

In [4]:
headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

# From here on I created the actual functions

This function goes through a playlist and retrieves:
- The artist
- The song name
- The song ID (this is needed for the next function)
- Song duration (in ms)
- Popularity (might be skewed towards the more recent songs though)
    - "According to Spotify, “popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are."" (Philip Peker, 2021) https://towardsdatascience.com/predicting-popularity-on-spotify-when-data-needs-culture-more-than-culture-needs-data-2ed3661f75f1

In [5]:
def playlist_songs(ID):
    r_playlist = requests.get(BASE_URL + 'playlists/' + ID, headers=headers)

    #looks at the specific songs within the playlist
    r2 = r_playlist.json()
    r3 = r2['tracks']
    r3.keys()
    r4 = r3['items']

    #puts results in dictionary
    data = []
    for r in r4:
        song_name = r['track']['name']
        song_url = r['track']['external_urls']['spotify']
        song_artist = r['track']['album']['artists'][0]['name']
        song_duration = r['track']['duration_ms']
        song_popularity = r['track']['popularity']
        song_year = r2['name'].split()[1]
        # only gives the last part after the '/' in the url, giving the song ID
        song_id  = song_url.split('/')[-1]
        #song_id is the key because two songs can have the same name
        data.append({
            'artist': song_artist,
            'song_name': song_name,
            'song_id': song_id,
            'song_popularity': song_popularity,
            'song_year': song_year
        })
    return data

The following function uses the playlist_songs function, and adds the audiofeatures to them
As a result we get:
- name, song and id
- audio features like
    - danceability
    - speechiness
    - tempo
    - etc.


In [6]:
def add_audiofeatures(playlist_id):
    playlist = playlist_songs(playlist_id)
    for song in playlist:
        track_id = song['song_id']
        trysong = requests.get(BASE_URL + 'audio-features/' + track_id, headers=headers)
        song.update(trysong.json())
    return playlist

In [7]:
euro2021 = add_audiofeatures("37i9dQZF1DWVCKO3xAlT1Q?si=236c30d415484365")
euro2020 = add_audiofeatures("0IyJJbmF3vp5nX6j4FFeDO?si=fc50f88c6eb54747")
euro2019 = add_audiofeatures("3ZdQUt8Tmtt7oOU8UM2koe?si=85c3bf9659eb4b4c")
euro2018 = add_audiofeatures("5sxwk5T34E2l2Ng02lipHS?si=600b20aa157f49da")
euro2017 = add_audiofeatures("6wECFlAQT9OTMCrdgGHPnz?si=219ac671992b4115")
euro2016 = add_audiofeatures("6ld9FVLMoZPxKezXf94AOK?si=8a1020be2e654345")
euro2015 = add_audiofeatures("0eW7WDU93NWFjDnKmqRelb?si=0e47c59fe0ba46e2")
euro2014 = add_audiofeatures("04U4zcFoZ8WqIDbhjnoxIv?si=cdf43bb693e74ae9")
euro2013 = add_audiofeatures("09OKlgunTlKElpxU6RHMQr?si=b28e0fc35d77413f")
euro2012 = add_audiofeatures("5tjy5qbWxaFFFTseRDptC7?si=8b010011750f4ab8")
euro2011 = add_audiofeatures("4HPdL5jwsxKnh7RTAwq9hF?si=59fc5fbb0bbe446c")
euro2010 = add_audiofeatures("3Vf3E8WavFCRLGF3a97nR9?si=f2114a3f8c6940d2")


# Quick look at what the dataframe will look like

In [22]:
pd.DataFrame(euro2010)

,artist,song_name,song_id,song_popularity,song_year,danceability,energy,key,loudness,mode,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Lena,Satellite,2UjNsh4ipNrmog6wdS2JVv,64,2010,0.680,0.857,11,-4.591,0,...,0.0414,0.924,94.928,audio_features,2UjNsh4ipNrmog6wdS2JVv,spotify:track:2UjNsh4ipNrmog6wdS2JVv,https://api.spotify.com/v1/tracks/2UjNsh4ipNrm...,https://api.spotify.com/v1/audio-analysis/2UjN...,175507,4
1,maNga,We Could Be The Same - Istanbul,1JzWdmqLRpqu2x6mfmxKjU,66,2010,0.661,0.887,2,-4.258,0,...,0.0441,0.745,133.018,audio_features,1JzWdmqLRpqu2x6mfmxKjU,spotify:track:1JzWdmqLRpqu2x6mfmxKjU,https://api.spotify.com/v1/tracks/1JzWdmqLRpqu...,https://api.spotify.com/v1/audio-analysis/1JzW...,183137,4
2,Ovi,Playing with Fire,0m3mkj4WTHkfrmHB6toUIK,36,2010,0.660,0.822,7,-2.683,1,...,0.0558,0.962,122.054,audio_features,0m3mkj4WTHkfrmHB6toUIK,spotify:track:0m3mkj4WTHkfrmHB6toUIK,https://api.spotify.com/v1/tracks/0m3mkj4WTHkf...,https://api.spotify.com/v1/audio-analysis/0m3m...,182951,4
3,Chanée & n'evergreen,In A Moment Like This,2wgVbKrFf3kf81FFuv53WP,10,2010,0.519,0.774,6,-5.254,1,...,0.1850,0.520,95.021,audio_features,2wgVbKrFf3kf81FFuv53WP,spotify:track:2wgVbKrFf3kf81FFuv53WP,https://api.spotify.com/v1/tracks/2wgVbKrFf3kf...,https://api.spotify.com/v1/audio-analysis/2wgV...,181293,4
4,Safura,Drip Drop,38hIyhovrQzznCiNjnG5LI,34,2010,0.468,0.657,6,-4.338,1,...,0.0776,0.228,147.965,audio_features,38hIyhovrQzznCiNjnG5LI,spotify:track:38hIyhovrQzznCiNjnG5LI,https://api.spotify.com/v1/tracks/38hIyhovrQzz...,https://api.spotify.com/v1/audio-analysis/38hI...,222493,3
5,Tom Dice,Me And My Guitar,2NN9Tsj56bSWq2cAklqsPB,0,2010,0.471,0.345,9,-8.373,1,...,0.1440,0.418,128.695,audio_features,2NN9Tsj56bSWq2cAklqsPB,spotify:track:2NN9Tsj56bSWq2cAklqsPB,https://api.spotify.com/v1/tracks/2NN9Tsj56bSW...,https://api.spotify.com/v1/audio-analysis/2NN9...,180940,4
6,Eva Rivas,Apricot Stone,2l3HUAMFX1c71GQJo71lT1,0,2010,0.605,0.673,11,-8.804,0,...,0.1610,0.379,101.058,audio_features,2l3HUAMFX1c71GQJo71lT1,spotify:track:2l3HUAMFX1c71GQJo71lT1,https://api.spotify.com/v1/tracks/2l3HUAMFX1c7...,https://api.spotify.com/v1/audio-analysis/2l3H...,181368,4
7,Sunstroke Project,Run Away - Radio Edit,06bUBaq3PKtjc6DvHN6uiX,0,2010,0.660,0.931,3,-3.038,0,...,0.1810,0.362,129.981,audio_features,06bUBaq3PKtjc6DvHN6uiX,spotify:track:06bUBaq3PKtjc6DvHN6uiX,https://api.spotify.com/v1/tracks/06bUBaq3PKtj...,https://api.spotify.com/v1/audio-analysis/06bU...,177539,4
8,Jessy Matador,Allez Ola Olé,1xSl7uE9De7aLDEGXahCJB,55,2010,0.642,0.934,11,-4.706,1,...,0.0612,0.721,129.024,audio_features,1xSl7uE9De7aLDEGXahCJB,spotify:track:1xSl7uE9De7aLDEGXahCJB,https://api.spotify.com/v1/tracks/1xSl7uE9De7a...,https://api.spotify.com/v1/audio-analysis/1xSl...,172746,4
9,Hera Björk,Je Ne Sais Quoi (Eurovision 2010) Orginal Version,48rIGviK0uiZKmXV6QifbE,30,2010,0.529,0.878,3,-4.328,0,...,0.1720,0.285,132.033,audio_features,48rIGviK0uiZKmXV6QifbE,spotify:track:48rIGviK0uiZKmXV6QifbE,https://api.spotify.com/v1/tracks/48rIGviK0uiZ...,https://api.spotify.com/v1/audio-analysis/48rI...,180001,4


# This creates the json files that can be used for analysis

In [9]:
with open("euro2021.json", "w") as outfile:
    outfile.write(json.dumps(euro2021))
with open("euro2020.json", "w") as outfile:
    outfile.write(json.dumps(euro2020))
with open("euro2019.json", "w") as outfile:
    outfile.write(json.dumps(euro2019))
with open("euro2018.json", "w") as outfile:
    outfile.write(json.dumps(euro2018))
with open("euro2017.json", "w") as outfile:
    outfile.write(json.dumps(euro2017))
with open("euro2016.json", "w") as outfile:
    outfile.write(json.dumps(euro2016))
with open("euro2015.json", "w") as outfile:
    outfile.write(json.dumps(euro2015))
with open("euro2014.json", "w") as outfile:
    outfile.write(json.dumps(euro2014))
with open("euro2013.json", "w") as outfile:
    outfile.write(json.dumps(euro2013))
with open("euro2012.json", "w") as outfile:
    outfile.write(json.dumps(euro2012))
with open("euro2011.json", "w") as outfile:
    outfile.write(json.dumps(euro2011))
with open("euro2010.json", "w") as outfile:
    outfile.write(json.dumps(euro2010))

## 